#### Lendo <mark>API do Banco Central</mark> para pegar os indicadores de Inflação dos últimos anos, calcular médias anuais e gravar em uma tabela no Lakehouse

In [5]:
import requests
import pandas as pd

tabela_final = pd.DataFrame()
pular_indice = 0
tamanho_pagina = 100000

while True:
    urlBase = f"https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$top={tamanho_pagina}&$skip={pular_indice}&$filter=Suavizada%20eq%20'S'&$format=json"
    print(urlBase)

    requisicao = requests.get(urlBase)
    informacoes = requisicao.json()
    tabela = pd.DataFrame(informacoes["value"])
    
    if len(informacoes['value']) < 1:
        break

    tabela_final = pd.concat([tabela_final, tabela])
    pular_indice += tamanho_pagina

# display(tabela_final)


# Filtrando dataFrame
df_filtrado = tabela_final[tabela_final['baseCalculo'] == 0][['Indicador', 'Data', 'Media']]

# Cria coluna Ano
df_filtrado['Data'] = pd.to_datetime(df_filtrado['Data'])
df_filtrado['Ano'] = df_filtrado['Data'].dt.year


# Agrupa Indicador e Ano para calcular a MediaAnual
media_por_ano = df_filtrado.groupby(['Indicador', 'Ano'])['Media'].mean().reset_index()
media_por_ano = media_por_ano.rename(columns={'Media': 'MediaAnual'})

# Filtrando pelos indicadores que vamos usar nessa POC, só para na apresentação o treinamento da célula seguinte andar mais rápido
filtro = media_por_ano['Indicador'].isin(['IPCA','IGP-M'])
media_por_ano = media_por_ano[filtro]
display(media_por_ano)


#Converte dataframe pandas para spark
dfMediaPorAno = spark.createDataFrame(media_por_ano)

# Salva tabela de médias dos indicadores por ano
# nome_tabela = "BC_Inflacao_Anual"
# spark_df.write.mode("overwrite").format("delta").save("Tables/" + nome_tabela)

StatementMeta(, 30b6eb70-029e-4ef6-94b9-566ab263d09b, 7, Finished, Available)

https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$top=100000&$skip=0&$filter=Suavizada%20eq%20'S'&$format=json
https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$top=100000&$skip=100000&$filter=Suavizada%20eq%20'S'&$format=json


/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


SynapseWidget(Synapse.DataFrame, 7e277470-0ee3-4a16-874c-fbc6c641193b)

#### Criando a predição dos indicadores para os próximos 10 anos e salvando em tabela

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Lista para armazenar os resultados
predicoes = []

# Iterar sobre cada indicador único
for indicador in media_por_ano['Indicador'].unique():
    
    # Filtrar os dados para o indicador atual
    subset = media_por_ano[media_por_ano['Indicador'] == indicador]
    
    # Preparar os dados
    X = subset['Ano'].values.reshape(-1, 1)  # Ano como variável independente
    y = subset['MediaAnual'].values           # Média Anual como variável dependente
    
    # Treinar o modelo
    model = LinearRegression().fit(X, y)
    
    # Prever para os próximos 10 anos
    anos_futuros = np.array(range(X[-1][0] + 1, X[-1][0] + 11)).reshape(-1, 1)
    predicoes_futuras = model.predict(anos_futuros)
    
    # Adicionar as predições à lista
    for ano, predicao in zip(anos_futuros, predicoes_futuras):
        predicoes.append({
            'Indicador': indicador,
            'Ano': ano[0],
            'MediaAnualPredita': predicao
        })

# Converter a lista de predições para um DataFrame
df_predicoes = pd.DataFrame(predicoes)

# Converte dataframe pandas para spark
dfPredicoes = spark.createDataFrame(df_predicoes)

# Salvando tabela com as predições dos próximos anos
# nome_tabela = "BC_Inflacao_Predicoes"
# spark_df.write.mode("overwrite").format("delta").save("Tables/" + nome_tabela)


display(df_predicoes)

StatementMeta(, 30b6eb70-029e-4ef6-94b9-566ab263d09b, 9, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


SynapseWidget(Synapse.DataFrame, 5dbbb0a4-7bd1-43b4-bd59-b92a7c87f64c)

Gravando as predições em uma tabela BC_Inflacao_Predicoes

In [10]:
# Unindo dataFrames e salvando no Lakehouse como tabela
dfInflacao = dfMediaPorAno.union(dfPredicoes)
display(dfInflacao)

# Salvando tabela com as predições dos próximos anos
nome_tabela = "BC_Inflacao"
dfInflacao.write.mode("overwrite").format("delta").save("Tables/" + nome_tabela)

StatementMeta(, 30b6eb70-029e-4ef6-94b9-566ab263d09b, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, b067a815-695f-4d85-a63d-7fc13ac46554)